# Gradient Descent Implementation

We use the public dataset on university admissions: Dataset http://www.ats.ucla.edu/stat/data/binary.csv -> here grades.txt

This dataset has three input features: GRE score, GPA, and the rank of the undergraduate school. 
Institutions with rank 1 have the highest prestige, those with rank 4 have the lowest.

In [2]:
import pandas as pd
import numpy as np
admissions = pd.read_csv('grades.txt')
admissions.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [3]:
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [4]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1


In [5]:
# 10 % Test data
np.random.seed(123)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

In [6]:
print(data)
print(test_data)

     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
234      1  1.837832  0.368135       1       0       0       0
42       1  0.106478 -0.630376       0       1       0       0
303      1  0.799020  1.550582       0       1       0       0
155      1 -0.066657 -1.392397       0       0       0       1
342      0 -0.759199  1.471752       0       0       0       1
..     ...       ...       ...     ...     ...     ...     ...
350      1  1.664697  1.603135       0       1       0       0
118      1  1.837832  0.814837       1       0       0       0
213      0  0.452749 -0.026014       0       0       1       0
353      0  0.972155  0.341859       0       1       0       0
358      1 -0.239793  0.788561       0       0       1       0

[360 rows x 7 columns]
     admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
2        1  1.837832  1.603135       1       0       0       0
17       0 -1.971146 -2.180695       0       0       1       0
32       0  0.106478  0.026539 

In [7]:
# Features and Labels / Targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidPrime(x):
    return sigmoid(x)*(1-sigmoid(x))

np.random.seed(123)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1/n_features**.5, size=n_features)

epochs = 1000
learnrate = 0.3

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        h=np.dot(x, weights)
        output = sigmoid(h)
        error = y - output
        error_term = error * sigmoidPrime(h)
        del_w += error_term * x

    weights += learnrate * del_w / n_records

    # Print error
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        e = np.mean((out - targets) ** 2)
        print("Error: %(e).6f" % {"e":(e)})


# Calculate accuracy
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Accuracy: %(acc).2f" % {"acc":(accuracy)})


Error: 0.266397
Error: 0.223190
Error: 0.209572
Error: 0.203310
Error: 0.200091
Error: 0.198311
Error: 0.197268
Error: 0.196627
Error: 0.196217
Error: 0.195947
Accuracy: 0.78
